In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt
import math
import os.path
import os
import csv
import glob
import cv2
import imageio
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as img
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import dlib
import shutil
from mlxtend.image import extract_face_landmarks
from imutils import face_utils
from threading import Thread
from scipy.spatial import distance 
from IPython.display import clear_output
from PIL import Image
import ssl

In [ ]:
def eye_aspect_ratio(eye):
    
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

def mouth_over_eye(eye):
    
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

In [ ]:
# ekstrak per 1 fps 
def frame_extraction(folder = "D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\VID\\", participants = [1, 60], MAX_FRAMES = 210):
    def getFrame(sec):
        
        start = 180000
        vidcap.set(cv2.CAP_PROP_POS_MSEC, start + sec*1000)
        hasFrames, image = vidcap.read()
        if hasFrames:
            if not (os.path.exists(r"D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Ekstrak FItur UTA-RLDD\\img\\" + str(participant) + "\\")):
                os.mkdir(r"D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Ekstrak FItur UTA-RLDD\\img\\" + str(participant))
            cv2.imwrite(r"D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Ekstrak FItur UTA-RLDD\\img\\" + str(participant) + "\p" + str(participant) + "_s" + str(state) +
                        "_" + str(count) + "sec.jpg", image)     # save frame
        return hasFrames, image

    participant, state = 0, 0
    # loop semua video
    for j in participants:
        for i in np.arange(0, 11, 10):
            clear_output(wait=True) #flush
            print(f'Partisipan {j} dan state {i}')
            data = []
            labels = []
            # mengambil direktori file dataset video
            vidcap = cv2.VideoCapture(r'D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\VID\\' + str(folder) + '\\' + str(j) +'\\' + str(i) + '.mov')
            print(f"Total Frame {int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))}")
            sec = 0
            frameRate = 1
            count = 0
            success, image  = getFrame(sec)
            participant = j
            state = i
            # ekstrak frame per video
            while success and count < MAX_FRAMES: 
                #grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                landmarks = extract_face_landmarks(image)
                if landmarks is not None and sum(sum(landmarks)) != 0:
                    count += 1
                    data.append(landmarks)
                    labels.append([i])
                    sec = sec + frameRate
                    success, image = getFrame(sec)
                    print(count)
                else:  
                    sec = sec + frameRate
                    success, image = getFrame(sec)
                    print("not detected")
            if success:
                data = np.array(data)
                labels = np.array(labels)
                print(f"Data {data.shape}")
                print(f"Labels {labels.shape}")
                features = []
                for d in data:
                    #facelandmark
                    eye = d[36:68]
                    ear = eye_aspect_ratio(eye)
                    mar = mouth_aspect_ratio(eye)
                    mouth_eye = mouth_over_eye(eye)
                    features.append([int(participant), ear, mar, mouth_eye])
                features = np.array(features)
                print(f"Features {features.shape}")
                # simpan ke csv
                np.savetxt('D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\csv\\' + str(folder) +'_features_'+str(participant)+'_'+str(state)+'.csv', features, delimiter = ",")
                np.savetxt('D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\csv\\' + str(folder) +'_labels_'+str(participant)+'_'+str(state)+'.csv', labels, delimiter = ",")
     

In [ ]:
%%time
# load frame dari video

x = os.listdir('D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\VID\\')
for i,j in zip(x, [os.listdir('D:\\Subek\\Kuliah\\UPN\\SKRIPSI\\Program Bismillah FIX BANGET\\Dataset UTA-RLDD\\VID\\'+i) for i in x]):
    frame_extraction(folder=i, participants=j)#3

In [ ]:
pd.read_csv(r"csv\\Fold5_part2_features_59_10.csv", header=None, names=["Participant", "EAR", "MAR", "MOE"])

In [ ]:
# merge setiap csv
df_features = [pd.read_csv(f, header=None, names=["Participant", "EAR", "MAR", "MOE"]) for f in glob.glob("csv\Fold*_features_*.csv")]
df_labels = [pd.read_csv(f, header=None, names=["Y"]) for f in glob.glob("csv\Fold*_labels_*.csv")]

df = pd.concat([pd.concat(df_features,ignore_index=True), pd.concat(df_labels,ignore_index=True)], axis=1)
print(df.shape) 
# Save 
df.to_csv(r'csv\merged\dataset.csv',index=False)
df.head()